In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from datetime import datetime
import time

# Import the API key.
from config import weather_api_key

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
coordinates

[(26.444581899689993, 32.9108557185223),
 (-43.33977064745222, -127.63098338165551),
 (5.437048024882415, 103.71745312842899),
 (9.605388827738764, -161.48213481219747),
 (-83.06401298576218, 142.00862379878208),
 (36.11577549450362, -114.35586255619803),
 (-46.04317862785948, 172.86507355812296),
 (12.0850645961077, -67.38040422776555),
 (43.57972978823511, -3.294219832619632),
 (-50.64216189033251, -168.30145188575446),
 (-73.74304096211591, 76.43906308360658),
 (-42.91126315094645, -163.1260410717846),
 (33.11252438048983, -138.9673543786309),
 (-42.5816049859171, -143.73788572084734),
 (22.94692831563657, -140.49957082592942),
 (-65.85878072898244, -77.33096084639303),
 (-69.9898861279846, -37.87876301993859),
 (-12.628514768461898, -145.38985110070053),
 (77.31731201428624, -85.76998770252247),
 (58.33283995050505, 83.05988432968911),
 (-35.86955630312853, -147.79277525068892),
 (-40.95829019736689, 154.72335735943813),
 (80.82760084915432, 15.167693228890784),
 (45.37555223851498

In [4]:
# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate_x, coordinate_y in coordinates:
    city = citipy.nearest_city(coordinate_x, coordinate_y).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count.
len(cities)

771

In [5]:
# Starting URL for Weather Map API Call.
url = "https://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [ ]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if i % 50 == 0 and i >= 50:
        set_count += 1
        record_count = 1
        time.sleep(60) # sleep for 1 minute to avoid timeout from API

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_country = city_weather["sys"]["country"]
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_description = city_weather["weather"][0]["description"]

        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | qena
Processing Record 2 of Set 1 | rikitea
Processing Record 3 of Set 1 | marang
Processing Record 4 of Set 1 | hilo
Processing Record 5 of Set 1 | hobart
Processing Record 6 of Set 1 | boulder city
Processing Record 7 of Set 1 | pareora
Processing Record 8 of Set 1 | kralendijk
Processing Record 9 of Set 1 | laredo
Processing Record 10 of Set 1 | vaini
Processing Record 11 of Set 1 | busselton
Processing Record 12 of Set 1 | avarua
Processing Record 13 of Set 1 | fortuna
Processing Record 14 of Set 1 | mataura
Processing Record 15 of Set 1 | punta arenas
Processing Record 16 of Set 1 | ushuaia
Processing Record 17 of Set 1 | tiarei
Processing Record 18 of Set 1 | qaanaaq
Processing Record 19 of Set 1 | kolpashevo
Processing Record 20 of Set 1 | batemans bay
Processing Record 21 of Set 1 | longyearbyen
Processing Record 22 of Set 1 | sarkand
Processing Record 23 of Set 1 | labuhan
Processing Rec

In [ ]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

In [ ]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")